In [1]:
from pyiron_atomistics import Project, pyiron_to_ase

In [2]:
pr = Project("formation_energy")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

https://next-gen.materialsproject.org/materials/mp-1067

# Al-Li Phase 

In [3]:
structure_AlLi = pr.create.structure.materialsproject.by_id("mp-1067")
pyiron_to_ase(structure_AlLi).write("LiAl_poscar", format="vasp")
len(structure_AlLi)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

4

In [4]:
structure_AlLi = pr.create.structure.ase.read("LiAl_poscar", format="vasp")
len(structure_AlLi)

4

In [5]:
job_AlLi = pr.create.job.Sphinx("spx_AlLi")
job_AlLi.structure = structure_AlLi
job_AlLi.set_kpoints([4,4,4])
job_AlLi.set_encut(600.0)
job_AlLi.server.cores = 1

In [6]:
murn_AlLi = pr.create.job.Murnaghan("murn_AlLi")
murn_AlLi.ref_job = job_AlLi
murn_AlLi.run()

The job murn_AlLi was saved and received the ID: 1451
The job murn_AlLi_0_9 was saved and received the ID: 1452
The job murn_AlLi_0_92 was saved and received the ID: 1453
The job murn_AlLi_0_94 was saved and received the ID: 1454
The job murn_AlLi_0_96 was saved and received the ID: 1455
The job murn_AlLi_0_98 was saved and received the ID: 1456
The job murn_AlLi_1_0 was saved and received the ID: 1457
The job murn_AlLi_1_02 was saved and received the ID: 1458
The job murn_AlLi_1_04 was saved and received the ID: 1459
The job murn_AlLi_1_06 was saved and received the ID: 1460
The job murn_AlLi_1_08 was saved and received the ID: 1461
The job murn_AlLi_1_1 was saved and received the ID: 1462


# Al Endmember

In [7]:
structure_Al = pr.create.structure.ase.bulk("Al", cubic=True)
len(structure_Al)

4

In [8]:
job_Al = pr.create.job.Sphinx("spx_Al")
job_Al.structure = structure_Al
job_Al.set_kpoints([4,4,4])
job_Al.set_encut(600.0)
job_Al.server.cores = 1

In [9]:
murn_Al = pr.create.job.Murnaghan("murn_Al")
murn_Al.ref_job = job_Al
murn_Al.run()

The job murn_Al was saved and received the ID: 1463
The job murn_Al_0_9 was saved and received the ID: 1464
The job murn_Al_0_92 was saved and received the ID: 1465
The job murn_Al_0_94 was saved and received the ID: 1466
The job murn_Al_0_96 was saved and received the ID: 1467
The job murn_Al_0_98 was saved and received the ID: 1468
The job murn_Al_1_0 was saved and received the ID: 1469
The job murn_Al_1_02 was saved and received the ID: 1470
The job murn_Al_1_04 was saved and received the ID: 1471
The job murn_Al_1_06 was saved and received the ID: 1472
The job murn_Al_1_08 was saved and received the ID: 1473
The job murn_Al_1_1 was saved and received the ID: 1474


# Li Endmember

In [10]:
structure_Li = pr.create.structure.ase.bulk("Li", cubic=True).repeat([2,2,2])
len(structure_Li)

16

In [11]:
job_Li = pr.create.job.Sphinx("spx_Li")
job_Li.structure = structure_Li
job_Li.set_kpoints([4,4,4])
job_Li.set_encut(600.0)
job_Li.server.cores = 1

In [12]:
murn_Li = pr.create.job.Murnaghan("murn_Li")
murn_Li.ref_job = job_Li
murn_Li.run()

The job murn_Li was saved and received the ID: 1475
The job murn_Li_0_9 was saved and received the ID: 1476
The job murn_Li_0_92 was saved and received the ID: 1477
The job murn_Li_0_94 was saved and received the ID: 1478
The job murn_Li_0_96 was saved and received the ID: 1479
The job murn_Li_0_98 was saved and received the ID: 1480
The job murn_Li_1_0 was saved and received the ID: 1481
The job murn_Li_1_02 was saved and received the ID: 1482
The job murn_Li_1_04 was saved and received the ID: 1483
The job murn_Li_1_06 was saved and received the ID: 1484
The job murn_Li_1_08 was saved and received the ID: 1485
The job murn_Li_1_1 was saved and received the ID: 1486


# Collect Results

In [31]:
def filter_funct(job):
    return len(job.job_name.split("_")) == 2

In [36]:
def get_element(job):
    return job.job_name.split("_")[1]

In [25]:
def get_energy(job):
    return job.content["output/equilibrium_energy"] / len(job.structure)

In [26]:
def get_volume(job):
    return job.content["output/equilibrium_volume"] / len(job.structure)

In [39]:
df = table.get_dataframe()
df

,job_id,name,energy,volume
0,1451,AlLi,-131.172432,16.118247
1,1463,Al,-57.200850,16.683757
2,1475,Li,-204.666601,20.310418


In [37]:
table = pr.create.table()
table.filter_function = filter_funct
table.add["name"] = get_element
table.add["energy"] = get_energy
table.add["volume"] = get_volume
table.convert_to_object = True
table.run(delete_existing_job=True)

The job table was saved and received the ID: 1487


Loading and filtering jobs:   0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

# Formation Energy

In [53]:
(2* df[df["name"]=="AlLi"]["energy"].values - (1* df[df["name"]=="Al"]["energy"].values + 1* df[df["name"]=="Li"]["energy"].values))/2

array([-0.23870682])

Materials Project: -0.208 eV/atom

# Lattice Constant

In [60]:
(df[df["name"]=="Al"]["volume"].values * 4) ** (1/3)

array([4.05618682])